# **Muziek voor de moderne mens**

__Groep:__ E2<br>
__Namen:__ David Kraakman, Haady Hassan, Sten Paffen, Niels Barnhoorn<br>
__URL GitHub online repository:__ https://github.com/David-Kraakman/DataVerhaal

## Introductie
Er wordt tegenwoordig dagelijks nieuwe muziek uitgebracht door verschillende artiesten. Onze dataset bevat alleen al gegevens van meer dan 278.000 nummers op Spotify, variërend van het jaar 1960 tot 2022. Al deze nummers hebben attributen zoals dansbaarheid, vrolijkheid, toonsoort, populariteit, energie en nog veel meer. Deze muziek kan door mensen worden beluisterd via de streamingdienst Spotify.<br>

Wat kan er gezegd worden over de verandering van muziek in de afgelopen jaren? Is muziek net zoveel veranderd als de mens? De mens is namelijk in bepaalde opzichten veranderd in de afgelopen jaren. De moderne technologische mens, ook wel de Homo Digitalis genoemd, is een mens die omringd is door technologie, waarvan de gevolgen nog niet helemaal duidelijk zijn (Valkenburg & Peter, 2007). Uit onderzoek naar Nederlandse jongeren blijkt dat deze moderne mens een verkort concentratievermogen heeft, een verslechterde nachtrust en mindere studieresultaten door de opkomst van social media in de afgelopen decennia (Kloosterman & Van Beuningen, 2015). Bovendien zijn de huidige Nederlandse jongeren minder gelukkig dan de jongeren van 20 jaar geleden(CBS, 2019). Ook in andere delen van de wereld zijn er verontrustende trends waargenomen. Zo is er in Amerika een grote stijging van angststoornissen onder mensen te zien (Goodwin et al., 2020). Aan de andere kant van dit verhaal is diezelfde moderne mens meer verbonden dan ooit door social media, is informatieoverdracht met behulp van nieuwe technologieën een stuk sneller dan voorheen en vindt er een grote globalisatie plaats.<br>

De mens is dus aan het veranderen, verandert muziek dan mee met deze mens? En de opkomst van internetgebruik, hoe manifesteert dit zich in de muziekindustrie?






## Dataset en Preprocessing<br>
### *Datasets*

1. main_dataset.csv https://www.kaggle.com/datasets/viktoriiashkurenko/278k-spotify-songs?select=main_dataset.csv
2. happiness_cantril_ladder.csv https://ourworldindata.org/grapher/happiness-cantril-ladder


De hoofddataset die in het verhaal wordt gebruikt, bevat informatie over meer dan 278.000 nummers op Spotify. De dataset is te verkrijgen door de eerste link te volgen en vervolgens enkel de main_dataset.csv te downloaden. Deze dataset wordt regelmatig bijgewerkt met behulp van de Spotify API. Nieuwe nummers worden toegevoegd en de API verzamelt verschillende attributen van nummers. Deze attributen omvatten onder andere populariteit, dansbaarheid, energie, toonsoort, luidheid, positiviteit en nog veel meer. Ook categorische attributen zoals artiestennaam, album naam en genre zijn beschikbaar in de database. Dankzij de vele gekwantificeerde attributen in de dataset kunnen berekeningen en vergelijkingen worden uitgevoerd op muzieknummers.<br>

De tweede kleinere dataset die gebruikt wordt gaat enkel over de gemiddelde levensvoldoening per land per jaar. Om deze te downloaden kan er op de site onder de wereldkaart op de download knop geklikt worden om de juiste dataset te downloaden.


### *Preprocessing*
__Spotify-dataset:__<br>
Voor het preprocessen van de dataset is er eerst een tijdsperiode vastgesteld van 1960 t/m 2022. Er waren namelijk erg weinig nummers uit de jaren voor 1960 in de dataset en het jaar 2023 was nog niet compleet. Vervolgens zijn rijen met Nan-waardes uit de dataset gehaald. Ook waren er bepaalde kolommen die een type 'str' hadden, maar eigenlijk het type 'list' of 'np.ndarray' moesten hebben. Deze kolommen zijn dan ook omgezet en tenslote is de kolom met de release datum van elk nummer nog omgezet van type 'str' naar type 'pd.DateTime'. Het uiteindelijke databestand is opgeslagen in een parquet-bestand. Dit bestandstype slaat namelijk ten opzichte van een CSV-bestand wél de objecttypes op van de dataset. Ook neemt een parquet-bestand een stuk minder ruimte in beslag dan een csv-bestand en valt het hierdoor binnen de 100mb bestandsgrootte restricties van GitHub.<br>

__Levensvoldoening-dataset:__<br>
De levensvoldoening-dataset vergde geen preprocessing. Het is een vrij kleine dataset en alles was hierin goed uitgewerkt.



In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import plotly.io as io

io.templates.default = 'plotly_dark'

main_dataset = pd.read_parquet('processed_dataset.parquet')
happiness_dataset = pd.read_csv('happiness_cantril_ladder.csv')

In [2]:
#@title Processing 1: Jaar toevoegen aan main_dataset
# Converteer de 'release_date' kolom naar een datetimewaarde
main_dataset['release_date'] = pd.to_datetime(main_dataset['release_date'], errors='coerce')

# Voeg een nieuwe kolom 'year' toe op basis van de 'release_date'
main_dataset['year'] = main_dataset['release_date'].dt.year

# Voeg een nieuwe kolom 'release_decade' toe op basis van de 'year'
main_dataset['release_decade'] = main_dataset['year'].map(lambda x: (x // 10)*10)

# Voeg een nieuwe kolom 'duration_s' toe op basis van de 'duration_ms'
main_dataset['duration_s'] = main_dataset['duration_ms'] // 1000

# Voeg een nieuwe kolom 'year_month' toe op basis van de 'release_date'
main_dataset['year_month'] = main_dataset['release_date'].dt.to_period('M')

In [20]:
# Groeperen op 'release_year' en het gemiddelde van 'duration_ms' per tien jaar berekenen
avg_duration_per_decade = main_dataset.groupby('release_decade')['duration_s'].mean().reset_index()

#Figuur 1 Aantal nieuwe nummers per jaar
fig_songs = go.Figure(data=go.Bar(x=main_dataset.groupby('year').count()['track_uri'].index,
                                  y=main_dataset.groupby('year').count()['track_uri'],
                                  legendgroup="group1",
                                  legendgrouptitle_text="Plot 1:",
                                  name = 'Aantal nieuwe nummers',
                                  marker_color=px.colors.qualitative.G10[7]))
fig_songs.update_layout(
    title='Aantal nieuwe nummers per jaar',
    xaxis=dict(title='Jaar'),
    yaxis=dict(title='Aantal nummers')
)
#Figuur 2 Gemiddelde muziekduur per Decennium
fig_duration = go.Figure(data=go.Bar(x=avg_duration_per_decade['release_decade'],
                                     y=avg_duration_per_decade['duration_s'],
                                     legendgroup="group2",
                                     legendgrouptitle_text="Plot 2:",
                                     name='Duratie nummers',
                                     marker_color=px.colors.qualitative.G10[8]))
fig_duration.update_layout(
    title='Gemiddelde duur nummer per tien jaar',
    xaxis=dict(title='Decennium'),
    yaxis=dict(title='Gemiddelde duur')
)

#Figuur 3 Muziekkenmerken
fig_stats = go.Figure()
for variable in ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']:
    fig_stats.add_trace(go.Scatter(x=main_dataset.groupby('year').mean(numeric_only=True)[variable].index,
                            y=main_dataset.groupby('year').mean(numeric_only=True)[variable],
                            mode="lines+text",
                            legendgroup="group3",
                            legendgrouptitle_text="Plot 3:",
                            name=variable,
                            text=[variable]))
fig_stats.update_traces(textposition='top center', marker=dict(color=px.colors.qualitative.T10))


#Figuren combineren tot één plot
fig_combined = sp.make_subplots(rows=3, cols=1, subplot_titles=['Aantal nieuwe nummers per Jaar', 'Gemiddelde duur nummer per tien jaar', 'Jaarlijkse gemiddelde waardes van aantal muziekkenmerken'])

#De traces bovenstaande traces toevoegen
fig_combined.add_trace(fig_songs.data[0], row=1, col=1)
fig_combined.add_trace(fig_duration.data[0], row=2, col=1)
for i in range(len(fig_stats.data)):
    fig_combined.add_trace(fig_stats.data[i], row=3, col=1)

#Update de x-assen van de subplots
fig_combined.update_xaxes(title_text="Jaar", row=1, col=1)
fig_combined.update_xaxes(title_text="Decennium", row=2, col=1)
fig_combined.update_xaxes(title_text="Jaar", row=3, col=1)

#Update de y-assen van de subplots
fig_combined.update_yaxes(title_text="Hoeveelheid nummers", row=1, col=1)
fig_combined.update_yaxes(title_text="Aantal seconden", row=2, col=1)
fig_combined.update_yaxes(title_text="Percentiel", row=3, col=1)

fig_combined.update_layout(height=1200, showlegend=True, legend={'groupclick':"toggleitem", 'itemdoubleclick':False})

fig_combined.show()


Om eerst wat meer inzicht te geven in de hoofddataset die gebruikt wordt in dit verhaal is hierboven een plot gemaakt met basisgegevens vanuit de dataset. In de eerste subplot kunnen we zien hoeveel nieuwe nummers er jaarlijks worden toegevoegd aan de Spotify catalogus en dat deze hoeveelheid exponentieel stijgt. In deze subplot staan namelijk op de x-as de jaartallen en op de y-as de hoeveelheid uitgebrachte muziek. Daaronder is in het tweede subplot te zien dat deze nieuwere muziek gemiddeld steeds korter van duur is. Op de x-as van het tweede subplot zijn dan ook de decennia geplot, terwijl op de y-as het gemiddelde aantal seconden dat een nummer in dat decennium duurde is geplot. In het laatste subplot worden de eerder genoemde attributen van muziek gevisualiseerd door de jaren heen. Op de x-as zijn wederom de jaartallen te zien en op de y-as zie je de percentielen. Alle muziek attributen zijn gemeten op een schaal van 0 tot 1 en in de derde plot is te zien hoe deze zich elk ontwikkelen door de jaren heen. Er is te zien dat de akoestiek van nummers na 1960 bijvoorbeeld daalt, terwijl de levendigheid vrij gelijk blijft. Met deze en nog meer variabelen probeert dit stuk de verandering van muziek te analyseren.


## Muziek verandert mee met de moderne mens

In [4]:
#@title Plot de verschuiving van mineur en majeur
mode_key_set = main_dataset[['mode', 'key', 'year']]
grouped_df = mode_key_set.groupby(['year', 'mode']).size().reset_index(name='count')
total_counts = grouped_df.groupby('year')['count'].sum()
grouped_df['percentiel'] = grouped_df['count'] / grouped_df['year'].map(total_counts)
grouped_df['mode'] = grouped_df['mode'].replace([1,0], ['Majeur', 'Mineur'])

# Dit is modus met een percentage
fig_mode = px.bar(grouped_df, x='year', y='percentiel', color='mode', barmode='stack',
                  labels={'year': 'Jaar', 'percentage': 'Percentage', 'mode': 'Modus'},
                  title='Percentage muziek in mineur en in majeur, per jaar',
                  color_discrete_sequence=px.colors.qualitative.T10)

# Plot de daling van valence over de jaren
fig_valence = px.scatter(main_dataset.groupby('year_month').mean(numeric_only=True),
                         x=main_dataset.groupby('year_month').mean(numeric_only=True).index.to_timestamp(),
                         y='valence',
                         title='Gemiddelde valence per maand, met trendlijn',
                         trendline="ols",
                         trendline_color_override = 'red')
fig_valence.update_layout(xaxis_title="Jaar",)

# Plot de gezamenlijke plot met mode en valence
fig_combined_2 = sp.make_subplots(rows=2, cols=1, subplot_titles=['Percentage muziek in mineur en in majeur, per jaar', 'Gemiddelde valence per maand, met trendlijn'])


#Voeg de traces toe van bovenstaande figuren
fig_combined_2.add_traces(fig_mode.data, rows=1, cols=1)
fig_combined_2.add_traces(fig_valence.data, rows=2, cols=1)
fig_combined_2.update_layout(barmode='stack')

#Update de x-axis labels
fig_combined_2.update_xaxes(title_text="Jaar", row=1, col=1)
fig_combined_2.update_xaxes(title_text="Jaar", row=2, col=1)



#Update de y-axis labels
fig_combined_2.update_yaxes(title_text="Percentage", row=1, col=1, tickformat=".0%")
fig_combined_2.update_yaxes(title_text="Percentiel", row=2, col=1)

fig_combined_2.update_layout(height=800, showlegend=True, legend={'groupclick':"toggleitem", 'itemdoubleclick':False})

fig_combined_2.show()

In de inleiding werd vermeld dat deze sombere moderne mens minder gelukkig is dan 20 jaar geleden volgens cijfers van het CBS (2019). Hoe zit dat met muziek? Vrolijke muziek wordt vaak in majeur geschreven en droevige muziek in mineur. In de bovenstaande staafdiagram wordt de verdeling getoond tussen nummers die in mineur zijn geschreven en de nummers die in majeur zijn geschreven. Op de x-as staat het aantal jaren en op y-as staat majeur en mineur verdeeld in percentages. Wat opvalt aan dit diagram is dat er over het algemeen niet veel veranderd is over de jaren heen. Vanaf de jaren ‘10 is er echter wel een lichte stijging te zien in de hoeveelheid nummers die in mineur zijn geschreven. Dit zou dus kunnen betekenen dat er tegenwoordig meer droevige muziek wordt uitgebracht dan voorheen.



Daarnaast is er ook iets wat opvalt bij de scatterplot met trendlijn over de gemiddelde 'valence' per maand. In deze subplot is op de x-as het jaar te zien en op de y-as is de valence-waarde tussen 0 en 1 te zien. De valence is de positiviteit of blijheid van een nummer gekwantificeerd door Spotify’s eigen API. Wat opvalt is het verschil in gemiddelde 'valance' van de jaren '60 en de jaren '10. De trendlijn toont namelijk een daling aan. Dit zou wederom kunnen betekenen dat er tegenwoordig meer droevige muziek wordt uitgebracht dan voorheen.

Deze daling van de 'valence' samen met de stijging van het aantal nummers dat geschreven is in een mineur-toonsoort zou erop kunnen duiden dat de minder gelukkige moderne mens over het algemeen in de afgelopen 60 jaar steeds meer sombere muziek is gaan luisteren en maken. Muziek lijkt mee te veranderen.



In [5]:
#@title Plot met blijheidsdata en correlaties
test_series = happiness_dataset.groupby('Year').mean(numeric_only=True)['Cantril ladder score']
test_series = test_series.map(lambda x: x/10)
#test_series = test_series.map(lambda x: (x - test_series.min())/(test_series.max() - test_series.min()))

test_series2 = main_dataset[(main_dataset['year'] >= 2003) & (main_dataset['year'] <= 2021)].groupby('year').mean(numeric_only=True)['valence']
#test_series2 = test_series2.map(lambda x: (x - test_series2.min())/(test_series2.max() - test_series2.min()))

correlation_pearson = test_series.corr(test_series2, method='pearson')
correlation_spearman = test_series.corr(test_series2, method='spearman')

fig_happiness_valence = go.Figure()

fig_happiness_valence.add_trace(go.Scatter(x= main_dataset[(main_dataset['year'] >= 2003) & (main_dataset['year'] <= 2021)].groupby('year').mean(numeric_only=True).index,
                                           y= test_series,
                                           name='Wereldwijde jaarlijkse levensvoldoening',
                                           mode = 'lines',
                                           line_color=px.colors.qualitative.T10[3]
                                           ))

fig_happiness_valence.add_trace(go.Scatter(x=main_dataset[(main_dataset['year'] >= 2003) & (main_dataset['year'] <= 2021)].groupby('year').mean(numeric_only=True).index,
                                           y=test_series2,
                                           name='Jaarlijkse gemiddelde valence',
                                           mode='lines',
                                           line_color=px.colors.qualitative.T10[2]
                                           ))


fig_happiness_valence.update_layout(title='Jaarlijkse gemiddelde levensvoldoening vs Jaarlijkse gemiddelde valence',
                                    xaxis_title="Jaar",
                                    yaxis_title="Percentiel",
                                    legend_title="Variabelen",)


fig_happiness_valence.add_annotation(text=f'Correlations:',
                                     align='left',
                                     showarrow=False,
                                     xref='paper',
                                     yref='paper',
                                     x=1.02,
                                     y=0.75,
                                     font_size = 14,
                                     bordercolor='black',
                                     borderwidth=1,
                                     xanchor="left")

fig_happiness_valence.add_annotation(text=f'Pearson Correlation: {correlation_pearson:.2f}',
                                     align='left',
                                     showarrow=False,
                                     xref='paper',
                                     yref='paper',
                                     x=1.03,
                                     y=0.7,
                                     bordercolor='black',
                                     borderwidth=1,
                                     xanchor="left")
fig_happiness_valence.show()

In het bovenstaande plot zijn de wereldwijde gemiddelde levensvoldoening en de valence van muziek tegenover elkaar geplot. De wereldwijde jaarlijkse levensvoldoening was eerst op een schaal van 1-10 gegeven, maar om het beter visueel te kunnen vergelijken met de valence is deze omgezet tot een schaal van 0-1. Op de x-as zijn de jaartallen vanaf 2004 te zien en op de y-as zijn dus de waarden tussen de 0 en 1 te zien van de twee verschillende variabelen. Ook is de Pearson correlatiecoëfficiënt tussen deze twee variabelen getoond. Hieruit blijkt dat er geen correlatie te vinden is tussen de twee variabelen. Het lijkt er dus op dat de gemiddelde levensvoldoening per jaar niet samenhangt met de gemiddelde valence per jaar.
Wat ook opvalt is dat de jaarlijkse gemiddelde levensvoldoening eigenlijk na een dip in 2004 redelijk stabiel blijft. Dit komt ook niet helemaal overeen met de bevindingen uit onze introductie dat de moderne mens steeds ongelukkiger wordt (CBS, 2019). Al met al ontwikkelen deze twee variabelen zich dus onafhankelijk van elkaar en kan er niet simpelweg gesteld worden dat meer sombere muziek leidt tot minder gelukkige mensen of andersom.


## De muziekindustrie en de opkomst van het internet

In [6]:
#@title Plot aantal unieke artiesten per jaar
# Converteer de 'artists_names'-kolom naar een tuple
main_dataset['artists_names'] = main_dataset['artists_names'].apply(tuple)

# Groepeer de dataset per jaar ('year') en tel het aantal unieke artiesten ('artists_names')
num_artists_per_year = main_dataset.groupby('year')['artists_names'].nunique().reset_index(name='num_artists')

#Maak de grafiek


fig_num_artists = go.Figure(go.Scatter(x=num_artists_per_year['year'],
                                       y=num_artists_per_year['num_artists'],
                                       mode='lines',
                                       fill='tozeroy',
                                       marker=dict(color=px.colors.qualitative.T10[1])))

fig_num_artists.update_layout(title='Jaarlijks aantal artiesten dat muziek heeft uitgebracht',
                              xaxis_title="Jaar",
                              yaxis_title="Hoeveelheid")
fig_num_artists.show()

In de bovenstaande plot is de hoeveelheid artiesten te zien die muziek hebben uitgebracht per jaar. Op de x-as zijn de jaartallen te zien en op de y-as is de hoeveelheid artiesten te zien. Wat al snel opvalt is dat de hoeveelheid artiesten dat muziek uitbrengt exponentieel toeneemt. Een mogelijke verklaring hiervoor is de opkomst van het internet. Deze grote verandering heeft het waarschijnlijk toegankelijker gemaakt voor (beginnende) artiesten om nieuwe muziek uit te brengen. Daarnaast zal de opkomst van muziekstreamingdiensten zoals Spotify hier waarschijnlijk aan hebben bijgedragen. Al met al lijkt het er dus op dat de opkomst van het internet een positieve invloed heeft gehad op de muziekindustrie door het gemakkelijker te maken om muziek uit te brengen.

In [7]:
#@title Create Artist-[Genres] Series
#Create series with structure: artists - genres_from_artists, for every song
artist_name_genres = main_dataset.loc[:, ['artists_names', 'artists_genres']]

#Create dict with structure: artist - [genres_from_artist]
artist_genres = dict()
for i in artist_name_genres.index:
    for name_index in range(len(artist_name_genres.loc[i, 'artists_names'])):
        artist_genres[artist_name_genres.loc[i, 'artists_names'][name_index]] = artist_name_genres.loc[i, 'artists_genres'][name_index]

#Create series from artist-genres dict
artist_genres_series = pd.Series(artist_genres)

In [8]:
#@title Create Year-Genre-Percentile Dataframe
year_genre = pd.DataFrame(columns=['year', 'genre', 'percentile'])

for year in range(1960, 2023, 1):
    #filter dataset on year
    artists_year = main_dataset[main_dataset['release_date'].dt.year == year]
    
    #create name_set based on artists in specific year
    name_set_year = set()
    for name_list in artists_year['artists_names']:
        for name in name_list:
            name_set_year = name_set_year | {name}

    #Create series with structure artist-[genres] from only specific year
    artist_genres_series_year = artist_genres_series.loc[list(name_set_year)]

    #Create set with every genre in the specific year
    genre_set_year = set()
    for i in artist_genres_series_year:
        for j in i:
            genre_set_year = genre_set_year | {j}

    #Make dict from the set with the count as values
    genre_dict_year = {genre:0 for genre in genre_set_year}

    #Count the amount of genres in the specific year and add them to the dict
    for i in artist_genres_series_year:
        for j in i:
            genre_dict_year[j] += 1

    amount_genres_year = sum([y[1] for y in sorted(genre_dict_year.items(), key=lambda x:x[1], reverse=True)])
    #Add most popular genre to series
    year_genre = pd.concat([year_genre, pd.DataFrame({'year':year,
                                                    'genre': [x[0] for x in sorted(genre_dict_year.items(), key=lambda x:x[1], reverse=True)[:5]],
                                                    'percentile': [(x[1]/amount_genres_year) for x in sorted(genre_dict_year.items(), key=lambda x:x[1], reverse=True)[:5]]})])

year_genre = year_genre.reset_index()

In [9]:
#@title Plot Graph: Top 5 meestgebruikte genres door artiesten, per jaar
# colors
color_sequence = px.colors.qualitative.Pastel + px.colors.qualitative.T10
colors = dict()
i = 0
for genre in [genre for genre in year_genre['genre'].unique()]:
    colors[genre] = (color_sequence)[i % len(color_sequence)]
    i += 1

# traces
data = []

# loop across the different years
for i in range(year_genre.shape[0]):

    #add a separate trace for each year, ordered from smallest to largest
    data.append(go.Bar(x=[year_genre.loc[i,'year']],
                        y=[year_genre.loc[i,'percentile']],
                        name=year_genre.loc[i,'genre'],
                        marker= dict(color=colors[year_genre.loc[i,'genre']]),
                        legendgroup=year_genre.loc[i,'genre'],
                        showlegend= year_genre.loc[i,'genre'] not in {data[x]['name'] for x in range(len(data))})) # show the legend only once for each trace

# layout
layout = dict(barmode='stack',
              yaxis={'title': 'percentage', 'tickformat' : "0%"},
              xaxis={'type': 'category', 'title': 'year'},
              title='Top 5 meestgebruikte genres door artiesten, per jaar')

# figure
fig_genres = go.Figure(data=data, layout=layout)

fig_genres.show()

Deze toename van artiesten heeft echter niet alleen positieve kanten. In het bovenstaande plot is een grote versplintering van de muziekgenres te zien. Op de x-as staan de jaartallen en op de y-as staan percentages. In het plot is gevisualiseerd wat de 5 meest gebruikte genres door artiesten van dat jaar waren en hoeveel procent deze innemen van het totale aantal genres van dat gegeven jaar. Aan het begin van de jaren ‘60 namen de 5 grootste genres nog meer dan 25% van de genres in, na het jaar 2000 nemen de top 5 genres nog maar rond de 5% in. Er is dus een flinke versplintering van het aantal genres waarin artiesten muziek maken.

In [10]:
#Create set with all genres in the genre column
genre_set = set()
for i in main_dataset['artists_genres']:
    for l in i:
        genre_set = genre_set | set(l)

In [11]:
#Split genres into words and count occurences of these words
genre_word_list = []
genre_word_set = set()
for genre in genre_set:
    words = genre.split()
    if ('hip' in words) and ('hop' in words):
        words.remove('hip')
        words.remove('hop')
        words.append('hip hop')
    genre_word_list = genre_word_list + words
    genre_word_set = genre_word_set | set(words)

#Create series with shape: Genre_word-Count(genre-word)
genreword_count_series = pd.Series({word:genre_word_list.count(word) for word in genre_word_set}).sort_values(ascending=False).loc[lambda x : x > 60]

#Create treemap from genreword_count_series
fig_treemap = go.Figure(go.Treemap(labels=genreword_count_series.index,
                                   values=genreword_count_series.values,
                                   parents=['']*len(genreword_count_series),
                                   textinfo = "label+value",
                                   marker= dict(colors=px.colors.qualitative.T10)))

fig_treemap.update_layout(title='Aantal muziekstijlen per genre', showlegend=True)
fig_treemap.show()


Deze versplintering wordt extra duidelijk in bovenstaande treemap diagram. In deze plot worden de genres die meer dan 60 soorten verschillende subgenres bevatten weergegeven. Denk hierbij aan ‘punk rock’ en ‘glam rock', die beide subgenres zijn van het hoofdgenre ‘rock’. In de plot is te zien dat voor het genre indie er dus al 437 subgenres te vinden zijn in de dataset. Voor mensen die muziek luisteren kan het door deze versplintering dus lastiger worden om het muzieklandschap te navigeren. Er zijn immers nu ontzettend veel verschillende soorten muziek(sub)genres waar uit gekozen kan worden. Muziek is dus toegankelijker geworden voor nieuwe artiesten door de opkomst van internet, maar tegelijkertijd is muziek onoverzichtelijk geworden en versplinterd.

# Conclusie

De mens is minder gelukkig (CBS, 2019), de valence van muziek daalt en het aandeel nummers in mineur stijgt voorzichtig. De mens en muziek lijken samen droeviger te worden, maar doordat er geen correlatie gevonden kan worden, blijken zij dit waarschijnlijk los van elkaar te doen. De opkomst van het internet heeft positieve en negatieve aspecten. De toegankelijkheid om muziek uit te brengen is verhoogd, maar muziekgenres zijn versplinterd en daardoor is het muzieklandschap minder overzichtelijk. Mens en muziek veranderen, dat is duidelijk geworden. Of dat goed is, is voor ieder zelf om te beslissen.

In [18]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Filteren van de dataset op criteria voor "happy" liedjes
happy_data = main_dataset[(main_dataset['valence'] > 0.6) &
                          (main_dataset['energy'] > 0.6) &
                          (main_dataset['danceability'] > 0.6)]


def plot_random_rows():
    clear_output()
    out = widgets.Output()

    button = widgets.Button(description='Genereer 5 willekeurige liedjes')
    button.layout.width = 'auto'
    button.on_click(lambda _: plot_random_rows())

    out.append_display_data(display(button))
    # random rows
    random_rows = happy_data.sample(n=5).loc[:, ['name', 'release_date', 'artists_names']]

    # tabel maken
    fig = go.Figure(data=[go.Table(header=dict(values=['Song-name', 'Release Date', 'Artists']),
                                   cells=dict(values=[random_rows['name'],
                                                      random_rows['release_date'].dt.date,
                                                      random_rows['artists_names']],
                                              fill_color='lavender',
                                              align='left',
                                              font=dict(color='black')))])

    # Weergeven van de output
    out.append_display_data(fig.show())

plot_random_rows()




Button(description='Genereer 5 willekeurige liedjes', layout=Layout(width='auto'), style=ButtonStyle())

# Bonus:
Na deze enigszins sombere conclusie van dit project, wilden we toch eindigen met een positieve noot. Hierboven vind je een aantal willekeurige nummers die allen een hoge mate van valence, energy en danceability hebben. Kortom, dit zijn nummers waar je blij van wordt. Geniet van de vrolijke muziek!

# Feedback 22-6-2023
Op donderdag 22 juni 2023 hebben we feedback gegeven en ontvangen. Uit de ontvangen feedback kwamen de volgende punten naar voren.


Wat betreft de barplot die de gemiddelde duur van een nummer per tien jaar weergeeft, kregen we van zowel onze TA als het andere groepje de opmerking dat het gemiddelde niet altijd veelzeggend is. Als voorbeeld werd gegeven dat als alle cijfers een 5 zouden zijn, het gemiddelde ook een 5 zou zijn. Maar als de helft van de cijfers een 1 is en de andere helft een 10, zou het gemiddelde nog steeds een 5 zijn. Ons werd aangeraden om in plaats daarvan de standaarddeviatie te gebruiken of een boxplot van de barplot te maken. Daarnaast werd er aangeraden om de data  in seconden te geven in plaats van milliseconden.

Daarnaast was het niet altijd even duidelijk waar bepaalde grafieken voor stonden. Ons werd aanbevolen om over het algemeen duidelijke titels te gebruiken. Ook werd voorgesteld om bij de populairste artiesten eerder te kiezen voor het weergeven van het aantal artiesten dat actief was in een bepaald jaar, omdat dit meer zegt over de toegankelijkheid van muziek maken.

Ten slotte kregen we feedback dat we onze hoofdvragen duidelijker moeten beantwoorden en meer nadruk moeten leggen op sociale media, evenals het toevoegen van een extra database om onze bevindingen te ondersteunen

We hebben het eerste punt van de feedback proberen te verwerken, maar wanneer wij probeerden om de barplot om te zetten naar bijvoorbeeld een boxplot of een violinplot bleek dat de plot erg onoverzichtelijk werd. In de dataset bij de duratie van nummers zaten namelijk veel extreme uitschieters en deze werden meegeplot in de boxplot en violinplot. Dit gaf een erg groot bereik op de y-as en dit leverde een onoverzichtelijke plot op. Wij vonden het echter moreel gezien niet juist om deze extreme outliers niet te tonen en daarom hebben we uiteindelijk er toch voor gekozen om enkel het gemiddelde te behouden. Voor het tweede punt hebben we de ontvangen feedback volledig toegepast op onze datavisualisatie, zoals te zien is in het eindresultaat. Na uitgebreid onderzoek hebben we ook het laatste feedbackpunt verbeterd door een extra database toe te voegen over het welzijn van mensen. Daarnaast hebben we wetenschappelijke artikelen toegevoegd om onze beweringen te ondersteunen.


# Bronnenlijst

De literatuurlijst is te vinden in de repository waarvan de link bovenaan deze pagina staat. Het bleek namelijk erg lastig om in markdown de literatuurlijst in apa-formaat getoond te krijgen. 

# Chat GPT verantwoording:

### Haady:

1:
Ik heb ChatGPT gebruikt als ondersteunend hulpmiddel bij het uitvoeren van gegevensverwerking op de 'main_dataset'. De reden voor het gebruik van ChatGPT was om snel en efficiënt codevoorbeelden te genereren die de gewenste bewerkingen op de dataset uitvoeren.

In dit specifieke geval heb ik de volgende ChatGPT prompts gebruikt om de Python-code te genereren die de volgende bewerkingen uitvoert op bij Processing1 ('main_dataset'):

    Converteer de 'release_date' kolom naar een datetimewaarde met behulp van de 'pd.to_datetime' functie en stel 'errors' in op 'coerce'.

    Voeg een nieuwe kolom genaamd 'year' toe aan 'main_dataset' door de 'dt.year' eigenschap van de 'release_date' kolom te gebruiken.

    Voeg een nieuwe kolom genaamd 'release_decade' toe aan 'main_dataset' door de 'year' kolom te mappen met behulp van een lambda-functie die elke waarde naar het decennium afrondt.

    Voeg een nieuwe kolom genaamd 'duration_s' toe aan 'main_dataset' door de 'duration_ms' kolom te delen door 1000.

    Voeg een nieuwe kolom genaamd 'year_month' toe aan 'main_dataset' door de 'dt.to_period' methode van de 'release_date' kolom te gebruiken en de frequentie in te stellen op 'M' voor maandelijks.

2:
Ik heb ChatGPT gebruikt om op een snelle en efficiënte manier Python-code te genereren om plots te maken bij: Plot aantal unieke artiesten per jaar.

Met behulp van de volgende prompts:

    Converteer de 'artists_names'-kolom naar een tuple met behulp van de 'apply' functie en de 'tuple' ingebouwde functie.

    Groepeer de gegevens per jaar ('year') en tel het aantal unieke artiesten ('artists_names') met behulp van de 'groupby' functie. Sla het resultaat op in de variabele 'num_artists_per_year'.

    Maak een lijndiagram met behulp van de 'go.Figure' functie van de Plotly bibliotheek. Gebruik de gegevens uit de 'num_artists_per_year' variabele om het jaarlijks aantal artiesten dat muziek heeft uitgebracht weer te geven. Stel de juiste titels en aslabels in.

3:
Ik heb ChatGPT gebruikt om op een snelle en efficiënte manier Python-code te genereren om plots te maken bij: Plot Graph: Aantal nummers per jaar Figuur 1 en 2

    Hoe kan ik de dataset groeperen op 'release_year' en het gemiddelde van 'duration_ms' per tien jaar berekenen

    Vervolgens wil ik twee grafieken maken: één met het aantal nieuwe nummers per jaar en één met de gemiddelde muziekduur per decennium?

### David

Ik heb geen ChatGPT gebruikt tijdens dit project. Ik heb mij voornamelijk beroepen op de documentatie van plotly en verschillende forums zoals stackoverflow.
### Sten

Ik heb geen gebruik gemaakt van ChatGPT.

### Niels

Ik heb ChatGPT gebruikt tijdens dit project, dit was voor het bedenken van ideeen voor verschillende invalshoeken voor dit project. Deze zijn niet gebruikt maar gaven wel aanstichting voor keuze.

    can you think of two perspectives when trying to make a data analyse for a data set about 270.000 songs with data such as liveness acousticness energy key?

Dit werd opgevolgd met verdere vragen over perspectieven zoals:

    can you think of a perspective that contains the influence of covid?

Destijds nog erg gefocust op covid en de invloeden van die periode op muziek. Dit mondde uit tot het huidige onderzoek. Het idee voor het huidige onderzoek komt dus niet direct voort uit ChatGPT.

# Taakverdeling

### Haady

Ik heb de basis van een aantal verschillende plots en subdatasets geschreven. Dit is later verbetered en geoptimaliseerd door David. Daarnaast heb ik de feedback/reflectie geschreven en de bonus tekst geschreven.

### Sten

Vinden DataBase en verantwoordelijk voor schriftelijk verhaal dataproject

### David

Hoofdverantwoordelijk voor de plots en de codes. Ook verantwoordelijk voor alles op GitHub krijgen. Ook alle preprocessing gedaan van de dataset. Daarnaast heb ik aan het einde geholpen met de geschreven tekst controleren en verbeteren.

### Niels

Gedeeltelijk verantwoordelijk voor geschreven tekst in het data verhaal en verdiepen in bronnen.